<a href="https://colab.research.google.com/github/alessandrotofani/Tesi_magistrale/blob/master/5_Dinamic_model_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
from google.colab import drive
drive.mount('/content/drive')
from sklearn.model_selection import train_test_split

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys 
sys.path.append('/content/drive/MyDrive/Tesi_magistrale/Tesi_magistrale')
import mf

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/data.csv')
data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

# Feature engineering and scaling

In [ ]:
data = mf.feature_engineering(data)
data = mf.feature_scaling(data)
data = pd.get_dummies(data)

# Dinamic model

In [ ]:
init_splits = 4
incoming_splits = 2
tot_splits = 10

In [ ]:
def get_split_id(data, init_splits, incoming_splits, tot_splits):
  id = {} # dizionario con (split : id_righe)
  id_fraud = {} # dizionario con (split: id righe transazioni fraudolente)
  for i in range(tot_splits):
    id[i] = data[(data['TransactionDT']>=(i/tot_splits)) & (data['TransactionDT']<((i + 1)/tot_splits))].index.tolist()
    id_fraud[i] = data[(data['TransactionDT']>=(i/tot_splits)) & (data['TransactionDT']<((i + 1)/tot_splits)) & (data['isFraud'] == 1)].index.tolist()
    
  return id, id_fraud

In [ ]:
class model:
  name = 'XGBoost'

  def __init__(self, t, model_dict):
    self.t = t
    self.f1 = 0
    self.precision = 0
    self.recall = 0
    self.roc = 0
    self.proba = []
    self.prediction = []
    self.train_id = [] # lista con le righe su cui il modello si è allenato
    self.val_id = [] # lista con le righe su cui il modello viene validato
    model_dict[t] = self
    return

  def __str__(self):
    return f"{name}, at time {self.t}, with f1 score of{self.f1}"

  def proba_predictions(self, X_val):
    self.proba = self.predict_proba(X_val, validate_features = False)
    return 

  def aggregate(self, soglia = 0.05):
    for proba in self.proba[:,1]:
      if proba > soglia:
        self.prediction.append(1)
      else:
        self.prediction.append(0)
    return

  def performance(self, soglia = 0.05):
    from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score
    mf.plot_cm(y_val, self.pred, 'Blues', p = soglia, dynamic = True)
    print('\n')
    print('Ensemble confusion matrix')

    self.score = f1_score(y_val, self.pred).round(2)
    self.roc = roc_auc_score(y_val, self.pred).round(2)
    self.precision = precision_score(y_val, self.pred,  average='binary').round(2)
    self.recall = recall_score(y_val, self.pred, average='binary').round(2)

    print('\n')
    print('Ensemble f1 score: ', self.score)
    print('Ensemble model roc auc score: ', self.roc)
    print('Ensemble model recall: ', self.recall)
    print('Ensemble model precision: ', self.precision)
    print('\n')
    return 

In [ ]:
class ensemble(model): # child class di model
  name = 'Ensemble'

  @override 
  def proba_predictions(self, X_val, model_dict):
    proba = []
    for t in model_dict:
      proba.append(model_dict[t].predict_proba(X_val, validate_features = False)[:, 1])
    self.proba = np.mean(proba, axis = 0)
    return 
  
  @override
  def aggregate(self, soglia = 0.05):
    for proba in self.proba:
      if proba > soglia:
        self.prediction.append(1)
      else:
        self.prediction.append(0)
    return 



In [ ]:
def dinamic_model(data, id, id_fraud, incoming_splits, tot_splits):
  tmax = np.ceil((tot_splits) / incoming_splits)

  for t in range(tmax):
    selected_id = get_t_id(data, id, t)
    current_model = train_model(data, selected_id, id_fraud)

  return current_model